In [1]:
import sys

In [2]:
sys.path.append('/home/lucas/Documents/research/maier-saupe-lc-hydrodynamics/app/analysis/utilities')

In [6]:
import tensor_calculus as tc
import sympy as sy

## Make all relevant symbols/functions

In [11]:
vec_dim = 5

x, y, z = sy.symbols('x y z')
coords = (x, y)
# coords = (x, y, z) # uncomment for 3D
xi = tc.TensorCalculusArray([x, y, z])

Z, theta = sy.symbols(r'Z theta')   
A, B, C = sy.symbols(r'A B C')
alpha = sy.symbols(r'alpha')

Q_vec = tc.make_function_vector(vec_dim, 'Q_{}', coords)
Q0_vec = tc.make_function_vector(vec_dim, 'Q_{{0{} }}', coords)
Lambda_vec = tc.make_function_vector(vec_dim, r'\Lambda_{}', coords)
Lambda0_vec = tc.make_function_vector(vec_dim, r'\Lambda_{{0{} }}', coords)
delta_Q_vec = tc.make_function_vector(vec_dim, r'\delta\ Q_{}', coords)

phi_i = sy.Function(r'\phi_i')(*coords)
phi_j = sy.Function(r'\phi_j')(*coords)

## Make basis and use that to make symbols tensors

In [12]:
basis = []

basis.append(tc.TensorCalculusArray([[1, 0, 0],
                                     [0, 0, 0],
                                     [0, 0, -1]]))
basis.append(tc.TensorCalculusArray([[0, 1, 0],
                                     [1, 0, 0],
                                     [0, 0, 0]]))
basis.append(tc.TensorCalculusArray([[0, 0, 1],
                                     [0, 0, 0],
                                     [1, 0, 0]]))
basis.append(tc.TensorCalculusArray([[0, 0, 0],
                                     [0, 1, 0],
                                     [0, 0, -1]]))
basis.append(tc.TensorCalculusArray([[0, 0, 0],
                                     [0, 0, 1],
                                     [0, 1, 0]]))

In [13]:
Q = tc.make_tensor_from_vector(Q_vec, basis)
Q0 = tc.make_tensor_from_vector(Q0_vec, basis)
Lambda = tc.make_tensor_from_vector(Lambda_vec, basis)
Lambda0 = tc.make_tensor_from_vector(Lambda0_vec, basis)
delta_Q = tc.make_tensor_from_vector(delta_Q_vec, basis)
Phi_i = tc.make_basis_functions(phi_i, basis)
Phi_j = tc.make_basis_functions(phi_j, basis)

dLambda_label = r'\frac{{\partial\ \Lambda_{} }}{{\partial\ Q_{} }}'
dLambda_mat = tc.make_function_matrix(vec_dim, dLambda_label, coords)
dLambda = tc.make_jacobian_matrix_list(dLambda_mat, Phi_j)

## Calculate residual

In [15]:
vec_dim = len(Phi_i)
R_mean_field = sy.zeros(vec_dim, 1)
R_entropy = sy.zeros(vec_dim, 1)
E1 = sy.zeros(vec_dim, 1)

for i in range(vec_dim):
    R_mean_field[i, 0] = alpha * Phi_i[i].ip(Q)
    R_entropy[i, 0] = -Phi_i[i].ip(Lambda)
    E1[i, 0] = -tc.grad(Phi_i[i], xi).ip(tc.grad(Q, xi))

R_mean_field = sy.simplify(R_mean_field)
R_entropy = sy.simplify(R_entropy)
E1 = sy.simplify(E1)

In [16]:
display(R_mean_field)
display(R_entropy)
display(E1)

Matrix([
[alpha*(2*Q_0(x, y) + Q_3(x, y))*\phi_i(x, y)],
[              2*alpha*Q_1(x, y)*\phi_i(x, y)],
[              2*alpha*Q_2(x, y)*\phi_i(x, y)],
[alpha*(Q_0(x, y) + 2*Q_3(x, y))*\phi_i(x, y)],
[              2*alpha*Q_4(x, y)*\phi_i(x, y)]])

Matrix([
[(-2*\Lambda_0(x, y) - \Lambda_3(x, y))*\phi_i(x, y)],
[                    -2*\Lambda_1(x, y)*\phi_i(x, y)],
[                    -2*\Lambda_2(x, y)*\phi_i(x, y)],
[(-\Lambda_0(x, y) - 2*\Lambda_3(x, y))*\phi_i(x, y)],
[                    -2*\Lambda_4(x, y)*\phi_i(x, y)]])

Matrix([
[-2*Derivative(Q_0(x, y), x)*Derivative(\phi_i(x, y), x) - 2*Derivative(Q_0(x, y), y)*Derivative(\phi_i(x, y), y) - Derivative(Q_3(x, y), x)*Derivative(\phi_i(x, y), x) - Derivative(Q_3(x, y), y)*Derivative(\phi_i(x, y), y)],
[                                                                                                              -2*Derivative(Q_1(x, y), x)*Derivative(\phi_i(x, y), x) - 2*Derivative(Q_1(x, y), y)*Derivative(\phi_i(x, y), y)],
[                                                                                                              -2*Derivative(Q_2(x, y), x)*Derivative(\phi_i(x, y), x) - 2*Derivative(Q_2(x, y), y)*Derivative(\phi_i(x, y), y)],
[-Derivative(Q_0(x, y), x)*Derivative(\phi_i(x, y), x) - Derivative(Q_0(x, y), y)*Derivative(\phi_i(x, y), y) - 2*Derivative(Q_3(x, y), x)*Derivative(\phi_i(x, y), x) - 2*Derivative(Q_3(x, y), y)*Derivative(\phi_i(x, y), y)],
[                                                                                      

## Calculate Jacobian

In [17]:
vec_dim = len(Phi_i)
dR_mean_field = sy.zeros(vec_dim, vec_dim)
dR_entropy = sy.zeros(vec_dim, vec_dim)
dE1 = sy.zeros(vec_dim, vec_dim)

for i in range(vec_dim):
    for j in range(vec_dim):
        dR_mean_field[i, j] = alpha * Phi_i[i].ip(Phi_j[j])
        dR_entropy[i, j] = -Phi_i[i].ip(dLambda[j])
        dE1[i, j] = -tc.grad(Phi_i[i], xi).ip(tc.grad(Phi_j[j], xi))

dR_mean_field = sy.simplify(dR_mean_field) 
dR_entropy = sy.simplify(dR_entropy) 
dE1 = sy.simplify(dE1)

In [18]:
display(dR_mean_field)
display(dR_entropy)
display(dE1)

Matrix([
[2*alpha*\phi_i(x, y)*\phi_j(x, y),                                 0,                                 0,   alpha*\phi_i(x, y)*\phi_j(x, y),                                 0],
[                                0, 2*alpha*\phi_i(x, y)*\phi_j(x, y),                                 0,                                 0,                                 0],
[                                0,                                 0, 2*alpha*\phi_i(x, y)*\phi_j(x, y),                                 0,                                 0],
[  alpha*\phi_i(x, y)*\phi_j(x, y),                                 0,                                 0, 2*alpha*\phi_i(x, y)*\phi_j(x, y),                                 0],
[                                0,                                 0,                                 0,                                 0, 2*alpha*\phi_i(x, y)*\phi_j(x, y)]])

Matrix([
[(-2*\frac{\partial\ \Lambda_0 }{\partial\ Q_0 }(x, y) - \frac{\partial\ \Lambda_3 }{\partial\ Q_0 }(x, y))*\phi_i(x, y)*\phi_j(x, y), (-2*\frac{\partial\ \Lambda_0 }{\partial\ Q_1 }(x, y) - \frac{\partial\ \Lambda_3 }{\partial\ Q_1 }(x, y))*\phi_i(x, y)*\phi_j(x, y), (-2*\frac{\partial\ \Lambda_0 }{\partial\ Q_2 }(x, y) - \frac{\partial\ \Lambda_3 }{\partial\ Q_2 }(x, y))*\phi_i(x, y)*\phi_j(x, y), (-2*\frac{\partial\ \Lambda_0 }{\partial\ Q_3 }(x, y) - \frac{\partial\ \Lambda_3 }{\partial\ Q_3 }(x, y))*\phi_i(x, y)*\phi_j(x, y), (-2*\frac{\partial\ \Lambda_0 }{\partial\ Q_4 }(x, y) - \frac{\partial\ \Lambda_3 }{\partial\ Q_4 }(x, y))*\phi_i(x, y)*\phi_j(x, y)],
[                                                      -2*\frac{\partial\ \Lambda_1 }{\partial\ Q_0 }(x, y)*\phi_i(x, y)*\phi_j(x, y),                                                       -2*\frac{\partial\ \Lambda_1 }{\partial\ Q_1 }(x, y)*\phi_i(x, y)*\phi_j(x, y),                                                   

Matrix([
[-2*Derivative(\phi_i(x, y), x)*Derivative(\phi_j(x, y), x) - 2*Derivative(\phi_i(x, y), y)*Derivative(\phi_j(x, y), y),                                                                                                                      0,                                                                                                                      0,     -Derivative(\phi_i(x, y), x)*Derivative(\phi_j(x, y), x) - Derivative(\phi_i(x, y), y)*Derivative(\phi_j(x, y), y),                                                                                                                      0],
[                                                                                                                     0, -2*Derivative(\phi_i(x, y), x)*Derivative(\phi_j(x, y), x) - 2*Derivative(\phi_i(x, y), y)*Derivative(\phi_j(x, y), y),                                                                                                                      0,                             

## Automatic code generation

In [23]:
Q_list = list(Q_vec)
Q0_list = list(Q0_vec)
delta_Q_list = list(delta_Q_vec)
phi_list = list([phi_i, phi_j])
Lambda_list = list(Lambda_vec)
Lambda0_list = list(Lambda0_vec)
symbol_list = [alpha, Z, theta]
symbol_list_code = ["alpha", "Z", "theta"]

display(Q_list)
display(Q0_list)
display(delta_Q_list)
display(phi_list)
display(Lambda_list)
display(Lambda0_list)
display(symbol_list)

[Q_0(x, y), Q_1(x, y), Q_2(x, y), Q_3(x, y), Q_4(x, y)]

[Q_{00 }(x, y), Q_{01 }(x, y), Q_{02 }(x, y), Q_{03 }(x, y), Q_{04 }(x, y)]

[\delta\ Q_0(x, y),
 \delta\ Q_1(x, y),
 \delta\ Q_2(x, y),
 \delta\ Q_3(x, y),
 \delta\ Q_4(x, y)]

[\phi_i(x, y), \phi_j(x, y)]

[\Lambda_0(x, y),
 \Lambda_1(x, y),
 \Lambda_2(x, y),
 \Lambda_3(x, y),
 \Lambda_4(x, y)]

[\Lambda_{00 }(x, y),
 \Lambda_{01 }(x, y),
 \Lambda_{02 }(x, y),
 \Lambda_{03 }(x, y),
 \Lambda_{04 }(x, y)]

[alpha, Z, theta]

In [27]:
mat_dim = 3

dQ_vec = tc.TensorCalculusArray.zeros(vec_dim, mat_dim)
dQ0_vec = tc.TensorCalculusArray.zeros(vec_dim, mat_dim)
ddQ_vec = tc.TensorCalculusArray.zeros(vec_dim, mat_dim, mat_dim)
dphi_vec = sy.zeros(2, mat_dim)

for i in range(vec_dim):
    for j in range(mat_dim):
        dQ_vec[i, j] = Q_list[i].diff(xi[j])
        
for i in range(vec_dim):
    for j in range(mat_dim):
        dQ0_vec[i, j] = Q0_list[i].diff(xi[j])
        
for i in range(vec_dim):
    for j in range(mat_dim):
        for k in range(mat_dim):
            ddQ_vec[i, j, k] = Q_list[i].diff(xi[j]).diff(xi[k])
        
for i in range(2):
    for j in range(mat_dim):
        dphi_vec[i, j] = phi_list[i].diff(xi[j])
        
        
dQ_vec_list = dQ_vec.tolist()
dQ0_vec_list = dQ0_vec.tolist()
ddQ_vec_list = ddQ_vec.tolist()
dphi_vec_list = dphi_vec.tolist()
dLambda_mat_list = dLambda_mat.tolist()


display(dQ_vec_list)
display(dQ0_vec_list)
display(ddQ_vec_list)
display(dphi_vec_list)
display(dLambda_mat_list)

[[Derivative(Q_0(x, y), x), Derivative(Q_0(x, y), y), 0],
 [Derivative(Q_1(x, y), x), Derivative(Q_1(x, y), y), 0],
 [Derivative(Q_2(x, y), x), Derivative(Q_2(x, y), y), 0],
 [Derivative(Q_3(x, y), x), Derivative(Q_3(x, y), y), 0],
 [Derivative(Q_4(x, y), x), Derivative(Q_4(x, y), y), 0]]

[[Derivative(Q_{00 }(x, y), x), Derivative(Q_{00 }(x, y), y), 0],
 [Derivative(Q_{01 }(x, y), x), Derivative(Q_{01 }(x, y), y), 0],
 [Derivative(Q_{02 }(x, y), x), Derivative(Q_{02 }(x, y), y), 0],
 [Derivative(Q_{03 }(x, y), x), Derivative(Q_{03 }(x, y), y), 0],
 [Derivative(Q_{04 }(x, y), x), Derivative(Q_{04 }(x, y), y), 0]]

[[[Derivative(Q_0(x, y), (x, 2)), Derivative(Q_0(x, y), x, y), 0],
  [Derivative(Q_0(x, y), x, y), Derivative(Q_0(x, y), (y, 2)), 0],
  [0, 0, 0]],
 [[Derivative(Q_1(x, y), (x, 2)), Derivative(Q_1(x, y), x, y), 0],
  [Derivative(Q_1(x, y), x, y), Derivative(Q_1(x, y), (y, 2)), 0],
  [0, 0, 0]],
 [[Derivative(Q_2(x, y), (x, 2)), Derivative(Q_2(x, y), x, y), 0],
  [Derivative(Q_2(x, y), x, y), Derivative(Q_2(x, y), (y, 2)), 0],
  [0, 0, 0]],
 [[Derivative(Q_3(x, y), (x, 2)), Derivative(Q_3(x, y), x, y), 0],
  [Derivative(Q_3(x, y), x, y), Derivative(Q_3(x, y), (y, 2)), 0],
  [0, 0, 0]],
 [[Derivative(Q_4(x, y), (x, 2)), Derivative(Q_4(x, y), x, y), 0],
  [Derivative(Q_4(x, y), x, y), Derivative(Q_4(x, y), (y, 2)), 0],
  [0, 0, 0]]]

[[Derivative(\phi_i(x, y), x), Derivative(\phi_i(x, y), y), 0],
 [Derivative(\phi_j(x, y), x), Derivative(\phi_j(x, y), y), 0]]

[[\frac{\partial\ \Lambda_0 }{\partial\ Q_0 }(x, y),
  \frac{\partial\ \Lambda_0 }{\partial\ Q_1 }(x, y),
  \frac{\partial\ \Lambda_0 }{\partial\ Q_2 }(x, y),
  \frac{\partial\ \Lambda_0 }{\partial\ Q_3 }(x, y),
  \frac{\partial\ \Lambda_0 }{\partial\ Q_4 }(x, y)],
 [\frac{\partial\ \Lambda_1 }{\partial\ Q_0 }(x, y),
  \frac{\partial\ \Lambda_1 }{\partial\ Q_1 }(x, y),
  \frac{\partial\ \Lambda_1 }{\partial\ Q_2 }(x, y),
  \frac{\partial\ \Lambda_1 }{\partial\ Q_3 }(x, y),
  \frac{\partial\ \Lambda_1 }{\partial\ Q_4 }(x, y)],
 [\frac{\partial\ \Lambda_2 }{\partial\ Q_0 }(x, y),
  \frac{\partial\ \Lambda_2 }{\partial\ Q_1 }(x, y),
  \frac{\partial\ \Lambda_2 }{\partial\ Q_2 }(x, y),
  \frac{\partial\ \Lambda_2 }{\partial\ Q_3 }(x, y),
  \frac{\partial\ \Lambda_2 }{\partial\ Q_4 }(x, y)],
 [\frac{\partial\ \Lambda_3 }{\partial\ Q_0 }(x, y),
  \frac{\partial\ \Lambda_3 }{\partial\ Q_1 }(x, y),
  \frac{\partial\ \Lambda_3 }{\partial\ Q_2 }(x, y),
  \frac{\partial\ \Lambda_3 }{\partial\ Q_3

In [36]:
from sympy.printing.cxx import CXX11CodePrinter
import re

In [37]:
class MyPrinter(CXX11CodePrinter):
    """
    This printer prints expressions as C++ code, and appropriately formats
    symbols that are specific to the nematic Q-tensor finite element simulation
    """
    
    def _print_Function(self, function):
        
        if function in Q_list:
            idx = Q_list.index(function)
            return self._print('Q_vec[q][{}]'.format(idx))
        if function in Q0_list:
            idx = Q0_list.index(function)
            return self._print('Q0_vec[q][{}]'.format(idx))
        if function in phi_list:
            idx = phi_list.index(function)
            if idx == 0:
                return self._print('fe_values.shape_value(i, q)')
            elif idx == 1:
                return self._print('fe_values.shape_value(j, q)')
            else:
                raise ValueError("Index out of bounds for phi")
        if function in Lambda_list:
            idx = Lambda_list.index(function)
            return self._print('Lambda_vec[{}]'.format(idx))
        if function in Lambda0_list:
            idx = Lambda0_list.index(function)
            return self._print('Lambda0_vec[{}]'.format(idx))
        for i, sublist in enumerate(dLambda_mat_list):
            if function in sublist:
                j = sublist.index(function)
                return self._print('dLambda_dQ[{}][{}]'.format(i, j))
        
        return super()._print_Function(function)
        
    def _print_Derivative(self, derivative):
        
        for i, sublist in enumerate(dQ_vec_list):
            if derivative in sublist:
                j = sublist.index(derivative)
                return self._print('dQ[q][{}][{}]'.format(i, j))
        for i, sublist in enumerate(dQ0_vec_list):
            if derivative in sublist:
                j = sublist.index(derivative)
                return self._print('dQ0[q][{}][{}]'.format(i, j))
        for i, sublist in enumerate(ddQ_vec_list):
            for j, subsublist in enumerate(sublist):
                if derivative in subsublist:
                    k = subsublist.index(derivative)
                    return self._print('ddQ[q][{}][{}][{}]'.format(i, j, k))
        for i, sublist in enumerate(dphi_vec_list):
            if derivative in sublist:
                j = sublist.index(derivative)
                if i == 0:
                    return self._print('fe_values.shape_grad(i, q)[{}]'.format(j))
                elif i == 1:
                    return self._print('fe_values.shape_grad(j, q)[{}]'.format(j))
                else:
                    raise ValueError("Index out of bounds for dphi")
            
        return super()._print_Derivative(derivative)
    
    def _print_Symbol(self, symbol):
        
        if symbol in symbol_list:
            i = symbol_list.index(symbol)
            return self._print(symbol_list_code[i])
        return super()._print_Symbol(symbol)
    
    def _print_Pow(self, Pow):
        
        if Pow.args[1] == 2:
            return "({}) * ({})".format(self.doprint(Pow.args[0]), self.doprint(Pow.args[0]))
        return super()._print_Pow(Pow)
        

In [38]:
def format_term(term):
    
    my_printer = MyPrinter()
    term_str = my_printer.doprint(term)
    
    # Take care of value functions
    term_str = term_str.replace("*fe_values.shape_value", "\n * fe_values.shape_value")
    term_str = term_str.replace('(alpha*dt + 1)', '(alpha * dt + 1)\n ')
    
    # Take care of grad functions
    # Not at beginning, plus or minus, check for space, anything before fe_values.shape_grad
    grad_pattern = r'(?<!^)([+-])\s*(.*?)\*fe_values\.shape_grad'
    # Just add a newline before, regularize space
    grad_replace = r'\n \1 \2 * fe_values.shape_grad'
    term_str = re.sub(grad_pattern, grad_replace, term_str)
    
    return term_str
    
def print_term(term, n_spaces):
    
    # if term is 0, just omit in code
    if term == 0:
        return ""
    
    # print code and internally format spacing
    term_str = format_term(term)
    space_str = " " * n_spaces
    
    # indent properly
    term_str = '(' + term_str                            # leading parenthesis
    term_str = term_str.replace('\n', '\n ' + space_str) # each newline add `space_str` spaces
    term_str = space_str + ' ' + term_str                # add `space_str` + 1 spaces to beginning
    term_str += ')'                                      # ending parenthesis
    
    return term_str

In [39]:
assign_string = "    cell_rhs(i) +=\n"
alen = 8
space_str = ' '*alen

term_list = [R_mean_field, R_entropy, E1]
for i in range(vec_dim):
    if i == 0:
        print("if (component_i == {})".format(i))
    else:
        print("else if (component_i == {})".format(i))
        
    print(assign_string, end="")
    print(space_str + "(\n", end="")
    
    first_term = True
    for term in term_list:
        # print a '+' unless we're on the first term
        if (not first_term):
            print(space_str, '+')
        
        first_term = False
        print( print_term(sy.simplify(term[i]), alen) )

    print(space_str, ')', sep='')
    print(space_str, '* fe_values.JxW(q);', sep='')

if (component_i == 0)
    cell_rhs(i) +=
        (
         (alpha*(2*Q_vec[q][0] + Q_vec[q][3])
          * fe_values.shape_value(i, q))
         +
         (-(2*Lambda_vec[0] + Lambda_vec[3])
          * fe_values.shape_value(i, q))
         +
         (-2*dQ[q][0][0]*fe_values.shape_grad(i, q)[0] 
          - 2*dQ[q][0][1] * fe_values.shape_grad(i, q)[1] 
          - dQ[q][3][0] * fe_values.shape_grad(i, q)[0] 
          - dQ[q][3][1] * fe_values.shape_grad(i, q)[1])
        )
        * fe_values.JxW(q);
else if (component_i == 1)
    cell_rhs(i) +=
        (
         (2*alpha*Q_vec[q][1]
          * fe_values.shape_value(i, q))
         +
         (-2*Lambda_vec[1]
          * fe_values.shape_value(i, q))
         +
         (-2*dQ[q][1][0]*fe_values.shape_grad(i, q)[0] 
          - 2*dQ[q][1][1] * fe_values.shape_grad(i, q)[1])
        )
        * fe_values.JxW(q);
else if (component_i == 2)
    cell_rhs(i) +=
        (
         (2*alpha*Q_vec[q][2]
          * fe_values.shape_val

In [41]:
assign_string = "    cell_matrix(i, j) +=\n"
alen = 12
space_str = ' ' * alen

term_list = [-dR_mean_field, -dR_entropy, -dE1]
for i in range(vec_dim):
    for j in range(vec_dim):
        if (i == 0) and (j == 0):
            print("if (component_i == {} && component_j == {})".format(i, j))
        else:
            print("else if (component_i == {} && component_j == {})".format(i, j))

        print(assign_string, end="")
        print(space_str + "(\n", end="")
        
        nonzero_term_list = []
        for term in term_list:
            if not (term[i, j] == 0):
                nonzero_term_list.append(term)
        
        first_term = True
        for k, term in enumerate(nonzero_term_list):
            print( print_term(term[i, j], alen) )
            first_term = False
            if not k == (len(nonzero_term_list) - 1):
                print(space_str, '+')

        print(space_str + ")")
        print(space_str, '* fe_values.JxW(q);', sep='')

if (component_i == 0 && component_j == 0)
    cell_matrix(i, j) +=
            (
             (-2*alpha
              * fe_values.shape_value(i, q)
              * fe_values.shape_value(j, q))
             +
             (-(-2*dLambda_dQ[0][0] - dLambda_dQ[3][0])
              * fe_values.shape_value(i, q)
              * fe_values.shape_value(j, q))
             +
             (2*fe_values.shape_grad(i, q)[0]*fe_values.shape_grad(j, q)[0] 
              + 2 * fe_values.shape_grad(i, q)[1]*fe_values.shape_grad(j, q)[1])
            )
            * fe_values.JxW(q);
else if (component_i == 0 && component_j == 1)
    cell_matrix(i, j) +=
            (
             (-(-2*dLambda_dQ[0][1] - dLambda_dQ[3][1])
              * fe_values.shape_value(i, q)
              * fe_values.shape_value(j, q))
            )
            * fe_values.JxW(q);
else if (component_i == 0 && component_j == 2)
    cell_matrix(i, j) +=
            (
             (-(-2*dLambda_dQ[0][2] - dLambda_dQ[3][2])
         